<a href="https://colab.research.google.com/github/Gokul-K-19/capstone/blob/main/DeeperMLP.py" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ======================================================
# 1️⃣ Imports
# ======================================================
import tensorflow as tf
import numpy as np
import pandas as pd
import time
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print("TensorFlow version:", tf.__version__)

# ======================================================
# 2️⃣ Upload & Load Dataset
# ======================================================

from google.colab import files
uploaded = files.upload()

df = pd.read_csv("basic_augmented_dataset.csv")

print("\nColumns:", df.columns)
print("Dataset shape:", df.shape)

# ======================================================
# 3️⃣ Encode Gender (M/F → 0/1)
# ======================================================

if df["GENDER"].dtype == "object":
    df["GENDER"] = df["GENDER"].map({"M": 0, "F": 1})

# ======================================================
# 4️⃣ Separate Features & Target
# ======================================================

X = df.drop(columns=["GLUCOSE_LEVEL"])
y = df["GLUCOSE_LEVEL"].values.reshape(-1,1)

print("\nSamples:", X.shape[0])
print("Features:", X.shape[1])

# ======================================================
# 5️⃣ Train-Test Split + Scaling
# ======================================================

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# ======================================================
# 6️⃣ Build Optimized MLP Model
# Architecture: 4 → 32 → 16 → 1
# ======================================================

model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')


model.compile(optimizer='adam', loss='mse')

model.summary()

# ======================================================
# 7️⃣ Train Model + Measure Time
# ======================================================

start_time = time.time()
model.fit(X_train, y_train, epochs=80, batch_size=32, verbose=1)
training_time = time.time() - start_time

print("\nTraining Time:", round(training_time,2), "seconds")

# ======================================================
# 8️⃣ Evaluate Float Model
# ======================================================

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("\nFLOAT MODEL PERFORMANCE")
print("MAE:", mae)
print("RMSE:", rmse)
print("R2:", r2)

# ======================================================
# 9️⃣ Save Float Model & Check Size
# ======================================================

model.save("float_model.h5")
float_size = os.path.getsize("float_model.h5") / 1024
print("\nFloat Model Size:", round(float_size,2), "KB")

# ======================================================
# 🔟 INT8 Quantization
# ======================================================

def representative_dataset():
    for i in range(min(100, len(X_train))):
        yield [X_train[i:i+1].astype(np.float32)]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_model = converter.convert()

with open("quant_model.tflite", "wb") as f:
    f.write(tflite_model)

quant_size = os.path.getsize("quant_model.tflite") / 1024
print("\nQuantized Model Size:", round(quant_size,2), "KB")

# ======================================================
# 1️⃣1️⃣ Evaluate Quantized Model
# ======================================================

interpreter = tf.lite.Interpreter(model_path="quant_model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_scale, input_zero_point = input_details[0]['quantization']
output_scale, output_zero_point = output_details[0]['quantization']

y_pred_quant = []
start_time = time.time()

for i in range(len(X_test)):
    x = X_test[i:i+1]

    x_quant = (x / input_scale + input_zero_point).astype(np.int8)

    interpreter.set_tensor(input_details[0]['index'], x_quant)
    interpreter.invoke()

    output = interpreter.get_tensor(output_details[0]['index'])
    output = (output.astype(np.float32) - output_zero_point) * output_scale

    y_pred_quant.append(output[0][0])

quant_time = time.time() - start_time

y_pred_quant = np.array(y_pred_quant).reshape(-1,1)

mae_q = mean_absolute_error(y_test, y_pred_quant)
rmse_q = np.sqrt(mean_squared_error(y_test, y_pred_quant))
r2_q = r2_score(y_test, y_pred_quant)

print("\nQUANTIZED MODEL PERFORMANCE")
print("MAE:", mae_q)
print("RMSE:", rmse_q)
print("R2:", r2_q)
print("Quantized Inference Time:", round(quant_time,2), "seconds")


TensorFlow version: 2.19.0


Saving basic_augmented_dataset.csv to basic_augmented_dataset.csv

Columns: Index(['AGE', 'GENDER', 'BMI', 'NIR', 'GLUCOSE_LEVEL'], dtype='object')
Dataset shape: (2000, 5)

Samples: 2000
Features: 4


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

 Trainable params: 1,761 (6.88 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/80
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 28952.4395
Epoch 2/80
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 26753.0254
Epoch 3/80
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 20628.5684
Epoch 4/80
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7062.3340
Epoch 5/80
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2443.3577
Epoch 6/80
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2203.7007
Epoch 7/80
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1606.7972
Epoch 8/80
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1454.1487
Epoch 9/80
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1364.9990
Epoch 10/80
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1202.2545
Epoch 11/80
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1203.1604
Epoch 12/80
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1214.9064
Epoch 13/80
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1158.9795
Epoch 14/80
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1209.1906
Epoch 15/80
50/50 ━━━━━━━━━━━━━━━━━━━━ 0


FLOAT MODEL PERFORMANCE
MAE: 20.95373841177561
RMSE: 29.41639253995638
R2: 0.8292295749404026

Float Model Size: 57.33 KB
Saved artifact at '/tmp/tmpgeh2re7r'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 4), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  132668236725136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132668236726096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132668236724176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132668236723408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132668236724944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132668236723792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132668236725328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132668236725904: TensorSpec(shape=(), dtype=tf.resource, name=None)

Quantized Model Size: 6.45 KB

QUANTIZED M

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [ ]:
print("Hello")